In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")

Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file: /home/qiany/yuehui/pyspark/Tetra_mesh/data/NASA_tetra/Lander_huge.ts



********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data/NASA_tetra


Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2): 2


tin_basename:  Lander_huge.ts
tin_filename:  Lander_huge
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


spark.sql.shuffle.partitions: 1600


In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

# set the Spark app name
spark_app_name = "Convert_ts2csv_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '200') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: Save_sorting_pts_sorting_tetra_Lander_big_06112024_2135


### read the TIN and save its vertices and triangles as csv files

In [ ]:
# process the TIN file in the format of .off
t0 = time.time()

url_pt_origin_csv = tin_directory + '/' + tin_filename + '_pts_origin.csv'
url_tetra_origin_csv = tin_directory + '/' + tin_filename + '_tetra_origin.csv'

if tin_extension == '.ts':
    with open(tin_file) as infile:
        # first line stores the number of vertices and tetrahedra
        line = (infile.readline()).split()
        vertices_num = int(line[0])
        tetra_num = int(line[1])
        print("vnum: {}, tnum:{}".format(vertices_num, tetra_num))
        with open(url_pt_origin_csv, 'w', newline='') as ofs_pts:
            writer = csv.writer(ofs_pts)
            for l in range(vertices_num):
                line = infile.readline().split()
                v = [float(line[0]),float(line[1]),float(line[2]),float(line[3]),l] # x,y,ele,self_index
                writer.writerow(v)
            ofs_pts.close()
            
        with open(url_tetra_origin_csv, 'w', newline='') as ofs_tetra:
            writer = csv.writer(ofs_tetra)
            tetra_order = 0
            for l in range(tetra_num):
                line = infile.readline().split()
                # in an off file, it is "3 235 32 532"
                tetrahedron = [int(line[0]),int(line[1]),int(line[2]),int(line[3]), tetra_order]        
                writer.writerow(tetrahedron)
                tetra_order += 1
            ofs_tetra.close()    
        infile.close()
    
t1 = time.time()
print("time cost:", t1-t0)

### sort the vertices of a mesh and get its filtration

In [11]:
# StructType is a collection of StructField’s which is used to define the column name, data type, and a flag for nullable or not.
# StructType: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/
schema_ver_origin = StructType([ \
    StructField("x",FloatType(),True), \
    StructField("y",FloatType(),True), \
    StructField("z",FloatType(),True), \
    StructField("ele",FloatType(),True), \
    StructField("self_index",IntegerType(),True) \
  ])

hdfs_tin_pts_origin = directory + "/" + tin_filename + '_pts_origin.csv'

df_ver_origin = spark.read.format("csv") \
      .option("header", False) \
      .schema(schema_ver_origin)\
      .load(hdfs_tin_pts_origin)

w = Window().orderBy(col('ele').asc())
df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
df_ver_order.printSchema()
# df_ver_order.show(5)

root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: integer (nullable = true)
 |-- self_order: integer (nullable = true)



### write the DataFrame to HDFS

In [12]:
t0 = time.time()

hdfs_tin_pts_filtra = directory + "/" + tin_filename + '_filtra_pts_sort.csv'
# df_ver_order.write.options(header='False', delimiter=',').csv("hdfs_data/sin_sum_VET_graph_node")
df_ver_order.write.csv(hdfs_tin_pts_filtra)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 4982.514730453491


In [15]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(directory, tin_filename):    
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
    hdfs_tin_tetra_origin = directory + "/" + tin_filename + '_tetra_origin.csv'
        
    schema_tetra_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("v4",IntegerType(),True)
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tin_tetra_origin)
    print("Number of partitions for df_tetra_origin:", df_tetra_origin.rdd.getNumPartitions())
        
    return df_tetra_origin

In [16]:
# read tetrahedra
df_tetra_origin = read_tetra_order(directory, tin_filename)
df_tetra_origin.printSchema()

Number of partitions for df_tetra_origin: 3031
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- v4: integer (nullable = true)



In [17]:
# replace_ver() replaces the original index of each vertex with filtra value in df_tetra_origin
def replace_ver(df_ver_order, df_tetra_origin):
    '''
    df_ver_order: a DataFrame storing sorted vertices with filtra values
    df_tetra_origin: a DataFrame storing triangles from a TIN
    '''
    df_tetra_order_v1 = df_tetra_origin.join(df_ver_order, df_tetra_origin.v1 == df_ver_order.self_index, "inner")
    df_tetra_order_v1 = df_tetra_order_v1.select(col("v1"), col("v2"), col("v3"), col("v4"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))

    df_tetra_order_v2 = df_tetra_order_v1.join(df_ver_order, df_tetra_order_v1.v2 == df_ver_order.self_index, "inner")
    df_tetra_order_v2 = df_tetra_order_v2.select(col("v1"), col("v2"), col("v3"), col("v4"),col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))

    df_tetra_order_v3 = df_tetra_order_v2.join(df_ver_order, df_tetra_order_v2.v3 == df_ver_order.self_index, "inner")
    df_tetra_order_v3 = df_tetra_order_v3.select(col("v1"), col("v2"), col("v3"), col("v4"),col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))

    df_tetra_order_v4 = df_tetra_order_v3.join(df_ver_order, df_tetra_order_v3.v4 == df_ver_order.self_index, "inner")
    df_tetra_order_v4 = df_tetra_order_v4.select(col("v1"), col("v2"), col("v3"), col("v4"),col("r1"), col("r2"), col("r3"),col("self_order").alias("r4"), col("r1_ele"), col("r2_ele"), col("r3_ele"),col("ele").alias("r4_ele"))

    df_tetra_order = df_tetra_order_v4.select(col("r1"), col("r2"), col("r3"), col("r4"), col("r1_ele"), col("r2_ele"), col("r3_ele"), col("r4_ele"))

    return df_tetra_order

In [18]:
# replaces original vertex index with filtra values
df_tetra_order = replace_ver(df_ver_order, df_tetra_origin)
df_tetra_order.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



In [19]:
t0 = time.time()

hdfs_tin_tetra_filtra = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'
# df_ver_order.write.options(header='False', delimiter=',').csv("hdfs_data/sin_sum_VET_graph_node")
df_tetra_order.write.csv(hdfs_tin_tetra_filtra)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 3799.469979763031
